In [8]:
import fitz  # PyMuPDF
import pandas as pd
import re  # Módulo de expressões regulares


In [9]:
def extrair_informacoes(pdf_path):
    doc = fitz.open(pdf_path)
    dados_faturas = []

    for pagina in doc:
        texto = pagina.get_text()

        # IDENTIFICAÇÃO DO EMITENTE
        identificacao_emitente_match = re.search(r"IDENTIFICAÇÃO DO EMITENTE[\s\S]*?(\n.+)", texto)
        identificacao_emitente = identificacao_emitente_match.group(1).strip() if identificacao_emitente_match else "Não encontrado"

        # Tentar extrair FATURA / DUPLICATA
        faturas_matches = re.findall(r"Vencimento:\s(\d{2}/\d{2}/\d{4})[\s\S]*?Valor:\sR\$\s([\d.]*\,\d{2})", texto)

        if faturas_matches:
            for data, valor in faturas_matches:
                valor_formatado = valor.replace(".", "").replace(",", ".")
                dados_faturas.append({
                    "IDENTIFICAÇÃO DO EMITENTE": identificacao_emitente,
                    "Data da fatura": data,
                    "Valor": valor_formatado
                })
        else:
            # Se não encontrar FATURA / DUPLICATA, procura DATA DE EMISSÃO e VALOR TOTAL DA NOTA
            data_emissao_match = re.search(r"DATA DE EMISSÃO[\s\S]*?(\d{2}/\d{2}/\d{4})", texto)
            data_emissao = data_emissao_match.group(1) if data_emissao_match else "Não disponível"
            
            # Expressão regular ajustada para captura do valor no formato "2.681,42"
            valor_total_nota_match = re.search(r"VALOR TOTAL DA NOTA[\s\S]*?([\d.]+,\d{2})", texto)
            if valor_total_nota_match:
                valor_total = valor_total_nota_match.group(1).replace(".", "").replace(",", ".")
                dados_faturas.append({
                    "IDENTIFICAÇÃO DO EMITENTE": identificacao_emitente,
                    "Data da fatura": data_emissao,
                    "Valor": valor_total
                })

    return dados_faturas


In [10]:
# Caminho para o seu arquivo PDF
pdf_path = "C:/Users/Administrativo/Downloads/nota_test.pdf"

# Extrair informações
dados_faturas = extrair_informacoes(pdf_path)

# Converter para DataFrame
df = pd.DataFrame(dados_faturas)

# Corrigindo o tipo de dado para 'Valor'
df["Valor"] = df["Valor"].astype(float)

# Exportar para Excel
nome_arquivo_excel = "C:/Users/Administrativo/Downloads/notas.xlsx"
df.to_excel(nome_arquivo_excel, index=False, float_format="%.2f")

print(f"Dados exportados para {nome_arquivo_excel} com sucesso.")

Dados exportados para C:/Users/Administrativo/Downloads/notas.xlsx com sucesso.
